<div class="clearfix" style="padding: 10px; padding-left: 0px">
<a href="https://www.unicamp.br/"><img src="http://www.fee.unicamp.br/sites/all/themes/theme537/images/unicamp.png" width="100px" style="display: inline-block; margin-top: 5px;"></a>
<a href="https://www.fee.unicamp.br/"><img src="http://www.fee.unicamp.br/sites/default/files/imagens/logo_feec_50anos_juntos.png" width="150px" class="pull-right" style="display: inline-block; margin: 0px;"></a>
</div>

## IE325 - Introduction to Reliability of Materials and Devices

### *Python Notebook Module for Reliability Characterization and Time to Failure Analysis*

<br>

|Authors   	|e-mail   	|
|:-:	|:-:	|
|Patricia Henriques Nallin  	|patricia.nallin@gmail.com   	| 
|Mauricio Martins Donatti  	|mmdonatti@gmail.com   	| 

<br>

The content of this module is licensed under the GNU General Public License v3.0.

---

In [7]:
# --------------------------------------------------------
# TIME TO FAILURE POWER LAW
# TF(csi, T) = Ao.(csi^-n).exp(Q/kT)
# 
#
# Linearization 1 - constant temperature
# ln(TF) = ln(Ao.exp(Q/kT)) - n.ln(csi)
# y = b + a.x
#
# Linearization 2 - constant strength (csi)
# ln(TF) = ln(Ao.csi^-n) + Q/k(T^-1)
# y = b + a.x
# --------------------------------------------------------
class TF_PowerLaw:
    
    # --------------------------------------------------------
    # Class Constructor
    # --------------------------------------------------------
    def __init__(self, mode, P1, P2, P3 = [], P4 = [], P5 = [], P6 = []):
        import numpy as np
        from scipy import stats

        # Variables and Operation Mode
        self.mode = mode
        if self.mode == 'all':
            Tcte_TF = P1
            Tcte_stress = P2
            Tcte = P3
            Scte_TF = P4
            Scte_temp = P5
            Scte = P6
        elif self.mode == 'csi':
            Tcte_TF = P1
            Tcte_stress = P2
        elif self.mode == 'temp':
            Scte_TF = P1
            Scte_temp = P2
        else:
            return        
        
        self.k = 8.617*10e-5 # Boltzmann constant => eV/K
        self.Q = 'not calculated'
        self.Ao = 'not calculated'
        self.n = 'not calculated'

        # ----- If csi is a variable or csi and temperature are both variables -----
        if self.mode != 'temp': 
            # ----- Data Linearization -----
            self.x_n = np.log(Tcte_stress)
            self.y_n = np.log(Tcte_TF)
            
            # ----- LINEAR FIT -----
            # Statistics
            slope, intercept, self.r_value_n, p_value, self.std_err_n = stats.linregress(self.x_n, self.y_n)
            # Numpy
            self.a_n, self.b_n = np.polyfit(self.x_n,self.y_n, deg=1)
            self.n = -self.a_n
            
        # ----- If temperature is a variable or csi and temperature are both variables -----    
        if self.mode != 'csi': 
            # ----- Data Linearization -----
            self.x_Q = 1/np.array(Scte_temp)
            self.y_Q = np.log(Scte_TF)
            
            # ----- LINEAR FIT -----
            # Statistics
            slope, intercept, self.r_value_Q, p_value, self.std_err_Q = stats.linregress(self.x_Q, self.y_Q)
            # Numpy
            self.a_Q, self.b_Q = np.polyfit(self.x_Q,self.y_Q, deg=1)
            self.Q = self.k*self.a_Q
        
        # ----- Only calculates Ao if both n and Q were found -----
        if self.mode == 'all':
            self.Ao = np.exp(self.b_n - (self.Q/(self.k*Tcte)))
        
        return
    
    # --------------------------------------------------------
    # Constants
    # --------------------------------------------------------
    def Variables(self):
        string_var = "-----\nLinear Fit Variables - Power Law\n\n"
        
        if self.mode != 'temp':         

            string_var += "Finding N exponent\n" + \
            "a (slope coef) = " + str(self.a_n)+ "\n"+ \
            "b (intercept coef) = " + str(self.b_n)+ "\n"+ \
            "Correlation coefficient = " + str(self.r_value_n)+ "\n"+ \
            "Standard error = " + str(self.std_err_n)+ "\n\n"
        
        if self.mode != 'csi':         

            string_var += "Finding Activation Energy (Q)\n" + \
            "a (slope coef) = " + str(self.a_Q)+ "\n"+ \
            "b (intercept coef) = " + str(self.b_Q)+ "\n"+ \
            "Correlation coefficient = " + str(self.r_value_Q)+ "\n"+ \
            "Standard error = " + str(self.std_err_Q)+ "\n\n\n"
        
        string_var += "-----\nTime to Failure Variables - Power Law\n"
        string_var += "n = " + str(self.n)+ "\n"+ \
        "Q = " + str(self.Q)+ " [eV]\n"+ \
        "Ao = " + str(self.Ao)+ "\n\n"
        
        
        return string_var
    
   # --------------------------------------------------------
    # String to write the TF Power Law.
    # decimal: number of decimal digits
    # --------------------------------------------------------
    def Law(self, decimal = 3):
        import numpy as np
        
        d = '.'+str(decimal)+'e'
        
        if self.mode == 'all':
            return "TF(csi, T) = " + format(self.Ao, d) + ".(csi)^(-" + format(self.n, d) + ").exp(" + format(self.Q/self.k, d) + "/T)" 
        
        elif self.mode == 'temp':
            return "TF(T) = " + format(np.exp(self.b_Q), d) + ".exp(" + format(self.Q/self.k, d) + "/T)" 
        
        elif self.mode == 'csi':
            return "TF(csi) = " + format(np.exp(self.b_n), d) + ".(csi)^(-" + format(self.n, d) + ")" 
    
    
    # --------------------------------------------------------
    # Calculate TF based on Power Law
    # TF(csi, T) = Ao.(csi^-n).exp(Q/kT)
    # --------------------------------------------------------
    def Find_TF(self, P1, P2=[]):
        import numpy as np
        
        # Operation Mode
        if self.mode == 'all':
            csi = P1
            T = P2
            return self.Ao*np.power(np.array(csi),-self.n)*np.exp(self.Q/(self.k*np.array(T)))
        
        elif self.mode == 'temp':
            T = P1
            return np.exp(self.b_Q)*np.exp(self.Q/(self.k*np.array(T)))
            
        elif self.mode == 'csi':
            csi = P1
            return np.exp(self.b_n)*np.power(np.array(csi),-self.n)
        

    # --------------------------------------------------------
    # Linearization of given data points - constant Temperature
    # --------------------------------------------------------   
    def LinearData_n(self):
        return self.x_n,self.y_n
    
    
    # --------------------------------------------------------
    # Linear fit curve of given data points  - constant Temperature
    # --------------------------------------------------------
    def LinearFit_n(self):
        self.LinearFit_n = self.a_n*self.x_n + self.b_n
        return self.LinearFit_n  
    
    
    # --------------------------------------------------------
    # Linearization of given data points - constant strength (csi)
    # --------------------------------------------------------
    def LinearData_Q(self):
        return self.x_Q,self.y_Q
    
    
    # --------------------------------------------------------
    # Linear fit curve of given data points  - constant strenght (csi)
    # --------------------------------------------------------
    def LinearFit_Q(self):
        self.LinearFit_Q = self.a_Q*self.x_Q + self.b_Q
        return self.LinearFit_Q  

In [8]:
# --------------------------------------------------------
# TIME TO FAILURE EXPONENTIAL LAW
# TF(csi, T) = Bo.exp(-gamma.csi).exp(Q/kT)
# 
#
# Linearization 1 - constant temperature
# ln(TF) = ln(Bo.exp(Q/kT)) - gamma.csi
# y = b + a.x
#
# Linearization 2 - constant strength (csi)
# ln(TF) = ln(Bo.exp(-gamma.csi)) + Q/k(T^-1)
# y = b + a.x
# --------------------------------------------------------
class TF_ExponentialLaw:
    
    # --------------------------------------------------------
    # Class Constructor
    # --------------------------------------------------------
    def __init__(self, mode, P1, P2, P3 = [], P4 = [], P5 = [], P6 = []):
        import numpy as np
        from scipy import stats

        # Variables and Operation Mode
        self.mode = mode
        if self.mode == 'all':
            Tcte_TF = P1
            Tcte_stress = P2
            Tcte = P3
            Scte_TF = P4
            Scte_temp = P5
            Scte = P6
        elif self.mode == 'csi':
            Tcte_TF = P1
            Tcte_stress = P2
        elif self.mode == 'temp':
            Scte_TF = P1
            Scte_temp = P2
        else:
            return       
        
        self.k = 8.617*10e-5 #Boltzmann constant => eV/K
        self.Q = 'not calculated'
        self.Bo = 'not calculated'
        self.gamma = 'not calculated'

        # ----- If csi is a variable or csi and temperature are both variables -----
        if self.mode != 'temp': 
            # ----- Data Linearization -----
            self.x_g = np.array(Tcte_stress)
            self.y_g = np.log(Tcte_TF)
            
            # ----- LINEAR FIT -----
            # Statistics
            slope, intercept, self.r_value_g, p_value, self.std_err_g = stats.linregress(self.x_g, self.y_g)
            # Numpy
            self.a_g, self.b_g = np.polyfit(self.x_g,self.y_g, deg=1)
            self.gamma = -self.a_g
            
            
        # ----- If temperature is a variable or csi and temperature are both variables -----    
        if self.mode != 'csi': 
            # ----- Data Linearization -----
            self.x_Q = np.array(1/np.array(Scte_temp))
            self.y_Q = np.log(Scte_TF)
            
            # ----- LINEAR FIT -----
            # Statistics
            slope, intercept, self.r_value_Q, p_value, self.std_err_Q = stats.linregress(self.x_Q, self.y_Q)
            # Numpy
            self.a_Q, self.b_Q = np.polyfit(self.x_Q,self.y_Q, deg=1)
            self.Q = self.k*self.a_Q
        
        # ----- Only calculates Bo if both gamma and Q were found -----
        if self.mode == 'all':
            self.Bo = np.exp(self.b_g - (self.Q/(self.k*Tcte)))
        
        return
    
    # --------------------------------------------------------
    # Constants
    # --------------------------------------------------------
    def Variables(self):
        string_var = "-----\nLinear Fit Variables - Exponential Law\n\n"
        
        if self.mode != 'temp':         

            string_var += "Finding gamma\n" + \
            "a (slope coef) = " + str(self.a_g)+ "\n"+ \
            "b (intercept coef) = " + str(self.b_g)+ "\n"+ \
            "Correlation coefficient = " + str(self.r_value_g)+ "\n"+ \
            "Standard error = " + str(self.std_err_g)+ "\n\n"
        
        if self.mode != 'csi':         

            string_var += "Finding Activation Energy (Q)\n" + \
            "a (slope coef) = " + str(self.a_Q)+ "\n"+ \
            "b (intercept coef) = " + str(self.b_Q)+ "\n"+ \
            "Correlation coefficient = " + str(self.r_value_Q)+ "\n"+ \
            "Standard error = " + str(self.std_err_Q)+ "\n\n\n"
        
        string_var += "-----\nTime to Failure Variables - Exponential Law\n"
        string_var += "gamma = " + str(self.gamma)+ "\n"+ \
        "Q = " + str(self.Q)+ " [eV]\n"+ \
        "Bo = " + str(self.Bo)+ "\n\n"
        
        
        return string_var
    
    # --------------------------------------------------------
    # String to write the TF Exponential Law.
    # decimal: number of decimal digits
    # --------------------------------------------------------
    def Law(self, decimal = 3):
        import numpy as np
        
        d = '.'+str(decimal)+'e'
        
        if self.mode == 'all':
            return "TF(csi, T) = " + format(self.Bo, d) + ".exp(-" + format(self.gamma, d) + ".csi).exp(" + format(self.Q/self.k, d) + "/T)" 
        
        elif self.mode == 'temp':
            return "TF(T) = " + format(np.exp(self.b_Q), d) + ".exp(" + format(self.Q/self.k, d) + "/T)" 
        
        elif self.mode == 'csi':
            return "TF(csi) = " + format(np.exp(self.b_g), d) + ".exp(-" + format(self.gamma, d) + ".csi)" 
    
    
    # --------------------------------------------------------
    # Calculate TF based on Exponential Law
    # TF(csi, T) = Bo.exp(-gamma.csi).exp(Q/kT)
    # --------------------------------------------------------
    def Find_TF(self, P1, P2=[]):
        import numpy as np
        
        # Operation Mode
        if self.mode == 'all':
            csi = P1
            T = P2
            return self.Bo*np.exp(-self.gamma*np.array(csi))*np.exp(self.Q/(self.k*np.array(T)))
        
        elif self.mode == 'temp':
            T = P1
            return np.exp(self.b_Q)*np.exp(self.Q/(self.k*np.array(T)))
            
        elif self.mode == 'csi':
            csi = P1
            return np.exp(self.b_g)*np.exp(-self.gamma*np.array(csi))
        

    # --------------------------------------------------------
    # Linearization of given data points - constant Temperature
    # --------------------------------------------------------   
    def LinearData_g(self):
        return self.x_g,self.y_g
    
    
    # --------------------------------------------------------
    # Linear fit curve of given data points  - constant Temperature
    # --------------------------------------------------------
    def LinearFit_g(self):
        self.LinearFit_g = self.a_g*self.x_g + self.b_g
        return self.LinearFit_g  
    
    
    # --------------------------------------------------------
    # Linearization of given data points - constant strength (csi)
    # --------------------------------------------------------
    def LinearData_Q(self):
        return self.x_Q,self.y_Q
    
    
    # --------------------------------------------------------
    # Linear fit curve of given data points  - constant strenght (csi)
    # --------------------------------------------------------
    def LinearFit_Q(self):
        self.LinearFit_Q = self.a_Q*self.x_Q + self.b_Q
        return self.LinearFit_Q  

In [9]:
# --------------------------------------------------------
# CHARACTERIZATION POWER LAW
# S = So(1-Ao.tˆn)
# 
# Linearization
# ln((S-So)/So) = ln(Ao) + m.ln(t)
# y = b + a.x
# --------------------------------------------------------

class PowerLaw:
    # --------------------------------------------------------
    # Class Constructor
    # --------------------------------------------------------
    def __init__(self, t, S, deltaT = 0):
        import numpy as np
        from scipy import stats
        self.S = S
        self.t = t
        self.deltaT = deltaT
        if self.deltaT:
            for i in range (0,len(self.t)):
                self.t[i] -= self.deltaT
        
        # ----- Data Linearization -----
        if self.S[0] > self.S[1]:
                self.lin_S = np.log((self.S[0]-np.array(self.S[1:]))/self.S[0])
        else:
                self.lin_S = np.log((np.array(self.S[1:])-self.S[0])/self.S[0])
        self.lin_t = np.log(np.array(self.t[1:]))

        
        # ----- LINEAR FIT -----
        # Statistics
        slope, intercept, self.r_value, p_value, self.std_err = stats.linregress(self.lin_t, self.lin_S)
        # Numpy
        self.a, self.b = np.polyfit(self.lin_t,self.lin_S, deg=1)

        self.Ao = np.exp(self.b)
        self.n = self.a
        
        if self.S[0] > self.S[1]:
            self.Ao = -self.Ao

        if self.deltaT:
            for i in range (0,len(self.t)):
                self.t[i] += self.deltaT

        return
    
    
    # --------------------------------------------------------
    # Calculate t, based on Power Law
    # S = So(1+Ao.tˆn) ----> t = (((S/So)-1)/Ao)^(1/n)
    # --------------------------------------------------------
    def Find_t(self, S):
        import numpy as np
        return np.power((((np.array(S)/self.S[0])-1)/self.Ao),(1/self.n)) + self.deltaT
    
    
    # --------------------------------------------------------
    # Calculate S, based on Power Law
    # S(t) = So(1+Ao.tˆn)
    # --------------------------------------------------------
    def Find_S(self, t):
        import numpy as np
        S = self.S[0]*(1 + self.Ao*(np.power(np.array(t)-self.deltaT,self.n)))
        return S
    
    
    # --------------------------------------------------------
    # Linearization of given data points 
    # --------------------------------------------------------
    def LinearData(self):
        return self.lin_t,self.lin_S
    
    
    # --------------------------------------------------------
    # Linear fit curve of given data points 
    # --------------------------------------------------------
    def LinearFit(self):
        self.LinearFit = []
        for i in range (len(self.lin_t)):
            self.LinearFit.append(self.a*self.lin_t[i] + self.b)
        return self.LinearFit
    
    
    # --------------------------------------------------------
    # String to write the Complete Power Law.
    # decimal: number of decimal digits
    # --------------------------------------------------------
    def LawComplete(self, decimal = 3):
        d = '.'+str(decimal)+'e'
        if self.deltaT:
            if self.Ao > 0:
                return str(self.S[0]) + " * (1 + " + format(self.Ao, d) + " * (t - " + str(self.deltaT) + ")^" + format(self.n, d) + ")"
            else:
                return str(self.S[0]) + " * (1 - " + format(-self.Ao, d) + " * (t - " + str(self.deltaT) + ")^" + format(self.n, d) + ")"
            
        else:
            if self.Ao > 0:
                return str(self.S[0]) + " * (1 + " + format(self.Ao, d) + " * t^" + format(self.n, d) + ")"
            else:
                return str(self.S[0]) + " * (1 - " + format(-self.Ao, d) + " * t^" + format(self.n, d) + ")"
        
        
    # --------------------------------------------------------
    # String to write the Simplified Power Law.
    # decimal: number of decimal digits
    # --------------------------------------------------------
    def LawSimple(self, decimal = 3):
        d = '.'+str(decimal)+'e'
        if self.deltaT:
            if self.Ao > 0:
                return str(self.S[0]) + " + " + format(self.S[0]*self.Ao, d) + " * (t - " + str(self.deltaT) + ")^" + format(self.n, d) + ")"
            else:
                return str(self.S[0]) + " - " + format(self.S[0]*-self.Ao, d) + " * (t - " + str(self.deltaT) + ")^" + format(self.n, d) + ")"

        else:
            if self.Ao > 0:
                return str(self.S[0]) + " + " + format(self.S[0]*self.Ao, d) + " * t^" + format(self.n, d)
            else:
                return str(self.S[0]) + " - " + format(self.S[0]*-self.Ao, d) + " * t^" + format(self.n, d)

    
    # --------------------------------------------------------
    # Derivate of Power Law
    # d(S)/dt = So.n.Ao.tˆ(n-1)
    # --------------------------------------------------------
    def Law_dSdt(self):
        return self.S[0]*self.n*self.Ao, self.n-1
    
    
    # --------------------------------------------------------
    # Derivate of Power Law
    # Previous element
    # --------------------------------------------------------
    def ddt_ant(self):
        dp = (np.subtract(self.S[1:],self.S[:-1]))/(np.subtract(self.t[1:],self.t[:-1]))
        dt = self.t[1:]
        return dt,dp
    
    
    # --------------------------------------------------------
    # Derivate of Power Law
    # Next element
    # --------------------------------------------------------
    def ddt_post(self):
        dp = (np.subtract(self.S[1:],self.S[:-1]))/(np.subtract(self.t[1:],self.t[:-1]))
        dt = self.t[:-1]
        return dt,dp

    
    # --------------------------------------------------------
    # Derivate of Power Law
    # Mean lateral elements
    # --------------------------------------------------------
    def ddt_med(self):
        dp = (np.subtract(self.S[2:],self.S[:-2]))/(np.subtract(self.t[2:],self.t[:-2]))
        dt = self.t[1:-1]
        return dt,dp
    
    
    # --------------------------------------------------------
    # Power Law Constants
    # --------------------------------------------------------
    def Variables(self):
        return "\n\nLinear Fit Variables - Power Law:\n\n" + \
        "a (slope coef) = " + str(self.a)+ "\n"+ \
        "b (intercept coef) = " + str(self.b)+ "\n"+ \
        "r_value (correlation coefficient) = " + str(self.r_value)+ "\n"+ \
        "std_err (standard error) = " + str(self.std_err)+ "\n"+\
        "\n\nPower Law Variables:\n\n" + \
        "Ao = " + str(self.Ao)    + "\n" + \
        "n = " + str(self.n)    + "\n"


In [10]:
# CHARACTERIZATION EXPONENTIAL LAW
# S = So.exp(Ao.t)
#
# Linearization
# ln(S/So) = Ao.t
# y = a.x

class ExponentialLaw:
    # --------------------------------------------------------
    # Class Constructor
    # --------------------------------------------------------
    def __init__(self, t, S, deltaT = 0):
        import numpy as np
        from scipy import stats
        self.S = S
        self.t = t
        self.deltaT = deltaT
        if self.deltaT:
            for i in range (0,len(self.t)):
                self.t[i] -= self.deltaT
        
        # ----- Data Linearization -----
        self.lin_S = np.log(np.array(self.S[1:])/self.S[0])
        self.lin_t = np.array(self.t[1:])
        
            
        # ----- LINEAR FIT -----

        # Statistics
        slope, intercept, self.r_value, p_value, self.std_err = stats.linregress(self.lin_t, self.lin_S)
        # Numpy
        self.a, self.b = np.polyfit(self.lin_t,self.lin_S, deg=1)

        self.Ao = self.a
    
        if self.deltaT:
            for i in range (0,len(self.t)):
                self.t[i] += self.deltaT
        return
    
    
    # --------------------------------------------------------
    # Calculate t, based on EXPONENTIAL-LAW
    # S(t) = So.exp(Ao.t) ----> t = ln((S/So))/Ao)
    # --------------------------------------------------------
    def Find_t(self, S):
        import numpy as np
        return (np.log(((np.array(S)/self.S[0]))/self.Ao)/self.Ao)+ self.deltaT
    
    
    # --------------------------------------------------------
    # Calculate S, based on EXPONENTIAL-LAW
    # S(t) = So.exp(Ao.t)
    # --------------------------------------------------------
    def Find_S(self, t):
        import numpy as np
        S = self.S[0]*np.exp(self.Ao*(np.array(t)-self.deltaT))
        return S
    
    
    # --------------------------------------------------------
    # Linearization of given data points
    # --------------------------------------------------------
    def LinearData(self):
        return self.lin_t,self.lin_S
    
    
    # --------------------------------------------------------
    # Linear fit curve of given data points 
    # --------------------------------------------------------
    def LinearFit(self):
        self.LinearFit = []
        for i in range (len(self.lin_t)):
            self.LinearFit.append(self.a*self.lin_t[i] + self.b)
        return self.LinearFit
    
    
    # --------------------------------------------------------
    # String to write the Exponential Law.
    # decimal: number of decimal digits
    # --------------------------------------------------------
    def Law(self, decimal = 3):
        d = '.'+str(decimal)+'e'
        if self.deltaT:
            return str(self.S[0]) + "*exp(" + format(self.Ao, d) + " *(t - " + str(self.deltaT) + "))"
        else:
            return str(self.S[0]) + "*exp(" + format(self.Ao, d) + " *t)"

            
    # --------------------------------------------------------
    # Derivate of Exponential Law
    # d(S)/dt = So.Ao.exp(Ao.t)
    # --------------------------------------------------------
    def Law_dSdt(self):
        return self.S[0]*self.Ao
    
    
    # --------------------------------------------------------
    # Exponential Law Constants
    # --------------------------------------------------------
    def Variables(self):
        return "\n\nLinear Fit Variables - Exponential Law:\n\n" + \
        "a (slope coef) = " + str(self.a)+ "\n"+ \
        "b (intercept coef) = " + str(self.b)+ "\n"+ \
        "r_value (correlation coefficient) = " + str(self.r_value)+ "\n"+ \
        "std_err (standard error) = " + str(self.std_err)+ \
        "\n\nExponential Law Variables:\n\n" + \
        "Ao = " + str(self.Ao)    

In [13]:
# --------------------------------------------------------
# Calculate data derivates in order to identify 
# degradation start time
# --------------------------------------------------------
def StartLaw(t, S):
    import numpy as np
    
    # Derivate regarding previous element
    ds_1 = (np.subtract(S[1:],S[:-1]))/(np.subtract(t[1:],t[:-1]))
    dt_1 = t[1:]

    # Derivate regarding next element
    ds_2 = (np.subtract(S[1:],S[:-1]))/(np.subtract(t[1:],t[:-1]))
    dt_2 = t[:-1]

    # Derivate regarding mean of lateral elements
    ds_3 = (np.subtract(S[2:],S[:-2]))/(np.subtract(t[2:],t[:-2]))
    dt_3 = t[1:-1]
    
    # Possibilities of data rate signal variation
    first_data_value = []
    first_data_value.append(dt_1[np.where(np.sign(ds_1) != np.sign(ds_1[-1]))[0][-1]])
    first_data_value.append(dt_2[np.where(np.sign(ds_2) != np.sign(ds_2[-1]))[0][-1]])
    first_data_value.append(dt_3[np.where(np.sign(ds_3) != np.sign(ds_3[-1]))[0][-1]])


    first_data_index = np.where(np.array(t) >= max(first_data_value))[0][0]

    return ds_1, dt_1, ds_2, dt_2, ds_3, dt_3, first_data_index